In [ ]:
from granite_io.backend.vllm_server import LocalVLLMServer

from granite_io.io.granite_3_3.input_processors.granite_3_3_input_processor import (
    Granite3Point3InputProcessor,
    Granite3Point3Inputs,
)

from granite_io.io.process_reward_model.best_of_n import ProcessRewardModelIOProcessor, PRMBestOfNCompositeIOProcessor

In [ ]:
# Spin up VLLM server for PRM

base_model_name = "ibm-granite/granite-3.3-8b-instruct"
lora_model_name = "/proj/checkpoints/aashka/models/reward_models/granite3.3-lora-gen-prm-v1"

# Start by firing up a local vLLM server and connecting a backend instance to it.
server = LocalVLLMServer(
    base_model_name, lora_adapters=[(lora_model_name, lora_model_name)]
)
server.wait_for_startup(200)
lora_backend = server.make_lora_backend(lora_model_name)
backend = server.make_backend()

In [ ]:
prm = ProcessRewardModelIOProcessor(backend=lora_backend)

In [ ]:
chat_input = Granite3Point3Inputs.model_validate(
    {
        "messages": [
            {"role": "user", "content": "What is the square root of 4x2x8"},
            {"role": "assistant", "content": "Lets first calculate the value of 4x2x8.\n\n 4x2 = 8 and 8x8=64 thus 4x2x8=64. \n\n The square root of 64 is 8. \n\n The answer is 8"},
        ],
    }
)

In [ ]:
outputs = await prm.acreate_chat_completion(chat_input)

In [ ]:
outputs

In [ ]:
if "server" in locals():
    server.shutdown()